#<h1><center><b> Apple-disease Resnet18 Pretrained </b></center></h1>

## Import

In [1]:
import os, sys, pdb, shutil, time, random
import argparse
import torch
import torch.backends.cudnn as cudnn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as func
import torchvision
from torchvision import transforms, datasets, models
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from shutil import copyfile
from os import listdir, makedirs, getcwd, remove
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.utils import make_grid
from skimage import io

## 1.Download Dataset

In [3]:
from google.colab import files
files.upload()

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d akshaygudi/apple-leaf-disease

 97% 790M/813M [00:04<00:00, 113MB/s]
100% 813M/813M [00:05<00:00, 170MB/s]


In [6]:
from zipfile import ZipFile
filename = "apple-leaf-disease.zip"

with ZipFile(filename,'r') as zip_dir:
    zip_dir.extractall()
    print("Done")

Done


In [ ]:
# !mkdir imag_data
# !mv train.csv imag_data/
# !mv images imag_data/

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# !mv test_data.csv imag_data/
# !mv train_data.csv imag_data/

##<h2>3. Custom Dataset<h2>

In [2]:
class MyDataset(Dataset):
    
    def __init__(self,csv_file,root_dir,transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img_path = os.path.join(self.root_dir,self.annotations.iloc[index,0])
#         print(img_path)
#         image = io.imread(img_path)
        image = Image.open(img_path).convert('RGB')
        ylabel = torch.tensor(int(self.annotations.iloc[index,-1]))
        
        if self.transform:
            image = self.transform(image)
        
        return (image,ylabel)        

## <h2>4. Initialization </h2>

In [3]:
args = {}

args["dataset"] = "apple"
args["model"] = "resnet18"
args["num_classes"] =  4
args["batch_size"] = 36
args["epochs"] = 70
args["learning_rate"] = 0.01
args["data_augmentation"] = True
args["cutout"] = True
args["n_holes"] = 1
args["length"] = 16
args["no_cuda"] = False
args["seed"] = 0

args["cuda"] = not args["no_cuda"] and torch.cuda.is_available()
cudnn.benchmark = True  

torch.manual_seed(args["seed"])
if args["cuda"]:
    torch.cuda.manual_seed(args["seed"])

test_id = args["dataset"] + '_' + args["model"]

print(args)

{'dataset': 'apple', 'model': 'resnet18', 'num_classes': 4, 'batch_size': 36, 'epochs': 70, 'learning_rate': 0.01, 'data_augmentation': True, 'cutout': False, 'n_holes': 1, 'length': 16, 'no_cuda': False, 'seed': 0, 'cuda': True}


## <h2>5. DataLoader and Transforms<h2>

### <h3>5.1 Cutout Regularization</h3>

In [4]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask
        
        return img

### <h3>5.2 Transform for Train Dataset</h3>

In [5]:
normalize_train = transforms.Normalize(mean = [0.4066, 0.5148, 0.3226],
                                      std=[0.2011, 0.1875, 0.1882])
train_transform = transforms.Compose([])

if args["data_augmentation"]:
    train_transform.transforms.append(transforms.RandomHorizontalFlip())

train_transform.transforms.append(transforms.Resize((224,224)))
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize_train)


if args["cutout"]:
    train_transform.transforms.append(Cutout(n_holes=args["n_holes"], length=args["length"]))

### <h3>5.3 Transform for Test Dataset</h3>

In [6]:
normalize_test = transforms.Normalize(mean = [0.4047, 0.5141, 0.3240],
                                     std=[0.2019, 0.1883, 0.1881])

test_transform = transforms.Compose([])
test_transform.transforms.append(transforms.Resize((224,224)))
test_transform.transforms.append(transforms.ToTensor())
test_transform.transforms.append(normalize_test)

### <h3>5.4 Create Data Loader</h3>

In [7]:
train_trans = train_transform
valid_trans = test_transform


train_set = MyDataset(csv_file="Apple_Leaf_Disease/train_data.csv",root_dir="Apple_Leaf_Disease/images/",transform=train_trans)
valid_set = MyDataset(csv_file="Apple_Leaf_Disease/test_data.csv",root_dir="Apple_Leaf_Disease/images/",transform=valid_trans)

        
train_loader = DataLoader(train_set, batch_size=args["batch_size"],shuffle=True)
test_loader = DataLoader(valid_set, batch_size=args["batch_size"], shuffle=False)

dataset_sizes = {
    'train': len(train_loader.dataset), 
    'valid': len(test_loader.dataset)
}

In [8]:
# i,j = train_set.__getitem__(2)

In [9]:
# results = transforms.ToPILImage()(i)
# results.show()

In [10]:
# i.shape

## <h2>6. MODELS</h2>

### <h3>6.1 Test Resnet working</h3>

In [11]:
use_gpu = torch.cuda.is_available()

resnet_18_model = models.resnet18(pretrained=True)
inputs, labels = next(iter(test_loader))
if use_gpu:
    resnet_18_model = resnet_18_model.cuda()
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())   
else:
    inputs, labels = Variable(inputs), Variable(labels)
outputs = resnet_18_model(inputs)
outputs.size()

torch.Size([36, 1000])

### <h3>6.2 Create Model</h3>

In [12]:
cnn = models.resnet18(pretrained=True)
args["num_classes"] = 4

# freeze all model parameters
for param in cnn.parameters():
    param.requires_grad = False

# New final layer with 4 classes
num_ftrs = cnn.fc.in_features
cnn.fc = torch.nn.Linear(num_ftrs, args["num_classes"]) 

if use_gpu:
    cnn = cnn.cuda()


criterion = nn.CrossEntropyLoss().cuda()
# cnn_optimizer = torch.optim.SGD(cnn.parameters(), lr=args["learning_rate"],
#                                 momentum=0.9, nesterov=True, weight_decay=5e-4)

cnn_optimizer = torch.optim.Adam(cnn.parameters(), lr=args["learning_rate"])

scheduler = MultiStepLR(cnn_optimizer, milestones=[5, 10], gamma=0.1)
print("Done - Configure Model")

Done - Configure Model


## <h2>7. Training</h2>

### <h3>7.1 Test </h3>

In [13]:
def test(loader):
    cnn.eval()
    correct = 0.
    total = 0.
    for images, labels in loader:
        images = images.cuda()
        labels = labels.cuda()

        with torch.no_grad():
            pred = cnn(images)

        pred = torch.max(pred.data, 1)[1]
        total += labels.size(0)
        correct += (pred == labels).sum().item()

    val_acc = correct / total
    cnn.train()
    return val_acc

### <h3>7.2 Training </h3>

In [14]:
def train_model(num_epochs):
  f = open("apple_resnet18_no_cutout_2.txt","a")
  for epoch in range(num_epochs):

      xentropy_loss_avg = 0.
      correct = 0.
      total = 0.

      progress_bar = tqdm(train_loader)
      cou = 0
      xentropy_loss_total = 0
      for i, (images, labels) in enumerate(progress_bar):
          progress_bar.set_description('Epoch ' + str(epoch))

          images = images.cuda()
          labels = labels.cuda()
                  
          cnn.zero_grad()
          pred = cnn(images)

          xentropy_loss = criterion(pred, labels)
          xentropy_loss.backward()
          cnn_optimizer.step()

          xentropy_loss_avg += xentropy_loss.item()
          xentropy_loss_total += xentropy_loss.item()*len(images)
          
          # Calculate running average of accuracy
          pred = torch.max(pred.data, 1)[1]
          total += labels.size(0)
          correct += (pred == labels.data).sum().item()
          accuracy = correct / total

          progress_bar.set_postfix(
              xentropy='%.3f' % (xentropy_loss_avg / (i + 1)),
              acc='%.3f' % accuracy)

      tqdm.write(' Running Tests.. ')
      test_acc = test(test_loader)
      total_loss_val = xentropy_loss_total/len(train_loader.dataset)
      tqdm.write('test_acc: %.3f' % (test_acc))
      tqdm.write('total loss : %f' % (total_loss_val))

      lr_rate = cnn_optimizer.state_dict()['param_groups'][0]['lr']
      tqdm.write('Learning rate: %f' % (lr_rate))

      r = "epoch: " + str(epoch) + ",  Train_acc: " + str(accuracy) + ",  Test_acc:  " + str(test_acc)

      f.write(r)
      f.write("\n")
      scheduler.step()
  f.close()

  # files.download('apple_resnet18_no_cutout_1.txt')

In [15]:
num_epochs = 15
train_model(num_epochs)

Epoch 0: 100%|██████████| 71/71 [02:19<00:00,  1.96s/it, acc=0.688, xentropy=0.945]


 Running Tests.. 


  0%|          | 0/71 [00:00<?, ?it/s]

test_acc: 0.804
total loss : 0.943250
Learning rate: 0.010000


Epoch 1: 100%|██████████| 71/71 [02:17<00:00,  1.94s/it, acc=0.767, xentropy=0.658]


...

 Running Tests.. 


  0%|          | 0/71 [00:00<?, ?it/s]

test_acc: 0.825
total loss : 0.353890
Learning rate: 0.000100


Epoch 14: 100%|██████████| 71/71 [02:17<00:00,  1.93s/it, acc=0.877, xentropy=0.347]


 Running Tests.. 
test_acc: 0.824
total loss : 0.345839
Learning rate: 0.000100


## <h1>Get Mean and Standard Deviation</h1>

In [ ]:
#Var(x) = E(x**2) - E(x)**2
import numpy as np


def get_mean_std(loader):
    channel_sum, channel_squared_sum, batches_val = 0, 0 ,0
    for data, _ in loader:
            channel_sum = channel_sum + torch.mean(data,dim=[0,2,3])
            channel_squared_sum = channel_squared_sum + torch.mean(data**2,dim=[0,2,3])
            batches_val = batches_val + 1
#             print("==",batches_val)
    mean = channel_sum/batches_val
    std =  (channel_squared_sum/batches_val - mean**2)**0.5
    return mean,std 

m,s = get_mean_std(train_loader)
print(m)
print(s)

## <h2>8. Unfreeze and Train</h2>

In [16]:
for name, child in cnn.named_children():
    if name in ['layer3', 'layer4','avgpool','fc']:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
            param.requires_grad = False

conv1 is frozen
bn1 is frozen
relu is frozen
maxpool is frozen
layer1 is frozen
layer2 is frozen
layer3 is unfrozen
layer4 is unfrozen
avgpool is unfrozen
fc is unfrozen


In [17]:
new_learning_rate = 0.01
cnn_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, cnn.parameters()), lr=new_learning_rate)

scheduler = MultiStepLR(cnn_optimizer, milestones=[10, 15], gamma=0.1)

In [ ]:
args["epochs"] = 70
train_model(args["epochs"])

Epoch 0: 100%|██████████| 71/71 [02:17<00:00,  1.93s/it, acc=0.870, xentropy=0.351]


 Running Tests.. 


  0%|          | 0/71 [00:00<?, ?it/s]

test_acc: 0.828
total loss : 0.351681
Learning rate: 0.000100


Epoch 1: 100%|██████████| 71/71 [02:16<00:00,  1.93s/it, acc=0.876, xentropy=0.347]


...

Epoch 53: 100%|██████████| 71/71 [02:17<00:00,  1.94s/it, acc=0.877, xentropy=0.338]


 Running Tests.. 


  0%|          | 0/71 [00:00<?, ?it/s]

test_acc: 0.832
total loss : 0.338524
Learning rate: 0.000100


Epoch 54:  25%|██▌       | 18/71 [00:34<01:43,  1.95s/it, acc=0.904, xentropy=0.320]